# Intro to Random Forests

## About this course

### Teaching approach

#### PXL Course
Our approach is simultaneously *bottom-up* and *top-down*. We start with some basic concepts *bottom-up*, because where you're coming from, you're missing some key basics. 
After these basics have been covered
* Supervised Learning
* Decision Trees
* Random Forests (concept)

We switch to a *top-down* approach. This is based on a part of the Fast AI Machine Learning Course, described below.

#### Fast AI
The Fast AI Machine Learning Course is taught by Jeremy Howard, and was developed by Jeremy along with Rachel Thomas.

This notebook will be using a *top-down* teaching method, which is different from how most math courses operate.  Typically, in a *bottom-up* approach, you first learn all the separate components you will be using, and then you gradually build them up into more complex structures.  The problems with this are that students often lose motivation, don't have a sense of the "big picture", and don't know what they'll need.

If you took the fast.ai deep learning course, that is what we used.  You can hear more about my teaching philosophy [in this blog post](http://www.fast.ai/2016/10/08/teaching-philosophy/) or [in this talk](https://vimeo.com/214233053).

Harvard Professor David Perkins has a book, [Making Learning Whole](https://www.amazon.com/Making-Learning-Whole-Principles-Transform/dp/0470633719) in which he uses baseball as an analogy.  We don't require kids to memorize all the rules of baseball and understand all the technical details before we let them play the game.  Rather, they start playing with a just general sense of it, and then gradually learn more rules/details as time goes on.

All that to say, don't worry if you don't understand everything at first!  You're not supposed to.  We will start using some "black boxes" such as random forests that haven't yet been explained in detail, and then we'll dig into the lower level details later.

To start, focus on what things DO, not what they ARE.

> Sam interjection: this is ok for things, but when it comes to people.... focus on who people ARE, not only on what they DO!


### Your practice

People learn by:
1. **doing** (coding and building)
2. **explaining** what they've learned (by writing or helping others)

Therefore, we suggest that you practice these skills on Kaggle by:
1. Entering competitions (*doing*)
2. Creating Kaggle kernels (*explaining*)

It's OK if you don't get good competition ranks or any kernel votes at first - that's totally normal! Just try to keep improving every day, and you'll see the results over time.

To get better at technical writing, study the top ranked Kaggle kernels from past competitions, and read posts from well-regarded technical bloggers. Some good role models include:

- [Peter Norvig](http://nbviewer.jupyter.org/url/norvig.com/ipython/ProbabilityParadox.ipynb) (more [here](http://norvig.com/ipython/))
- [Stephen Merity](https://smerity.com/articles/2017/deepcoder_and_ai_hype.html)
- [Julia Evans](https://codewords.recurse.com/issues/five/why-do-neural-networks-think-a-panda-is-a-vulture) (more [here](https://jvns.ca/blog/2014/08/12/what-happens-if-you-write-a-tcp-stack-in-python/))
- [Julia Ferraioli](http://blog.juliaferraioli.com/2016/02/exploring-world-using-vision-twilio.html)
- [Edwin Chen](http://blog.echen.me/2014/10/07/moving-beyond-ctr-better-recommendations-through-human-evaluation/)
- [Slav Ivanov](https://blog.slavv.com/picking-an-optimizer-for-style-transfer-86e7b8cba84b) (fast.ai student)
- [Brad Kenstler](https://hackernoon.com/non-artistic-style-transfer-or-how-to-draw-kanye-using-captain-picards-face-c4a50256b814) (fast.ai and USF MSAN student)

### Books

The more familiarity you have with numeric programming in Python, the better. If you're looking to improve in this area, we strongly suggest Wes McKinney's [Python for Data Analysis, 2nd ed](https://www.amazon.com/Python-Data-Analysis-Wrangling-IPython/dp/1491957662/ref=asap_bc?ie=UTF8).

For machine learning with Python, we recommend:

- [Introduction to Machine Learning with Python](https://www.amazon.com/Introduction-Machine-Learning-Andreas-Mueller/dp/1449369413): From one of the scikit-learn authors, which is the main library we'll be using
- [Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow, 2nd Edition](https://www.amazon.com/Python-Machine-Learning-scikit-learn-TensorFlow/dp/1787125939/ref=dp_ob_title_bk): New version of a very successful book. A lot of the new material however covers deep learning in Tensorflow, which isn't relevant to this course
- [Hands-On Machine Learning with Scikit-Learn and TensorFlow](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1491962291/ref=pd_lpo_sbs_14_t_0?_encoding=UTF8&psc=1&refRID=MBV2QMFH3EZ6B3YBY40K)


## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [ ]:
PATH = "../../data/aquaponics/"

In [ ]:
data = !ls {PATH}
data

### Fastai's teaching

At fast.ai we have a distinctive [teaching philosophy](http://www.fast.ai/2016/10/08/teaching-philosophy/) of ["the whole game"](https://www.amazon.com/Making-Learning-Whole-Principles-Transform/dp/0470633719/ref=sr_1_1?ie=UTF8&qid=1505094653).  This is different from how most traditional math & technical courses are taught, where you have to learn all the individual elements before you can combine them (Harvard professor David Perkins call this *elementitis*), but it is similar to how topics like *driving* and *baseball* are taught.  That is, you can start driving without [knowing how an internal combustion engine works](https://medium.com/towards-data-science/thoughts-after-taking-the-deeplearning-ai-courses-8568f132153), and children begin playing baseball before they learn all the formal rules.

### Fastai's approach to machine learning

Most machine learning courses will throw at you dozens of different algorithms, with a brief technical description of the math behind them, and maybe a toy example. You're left confused by the enormous range of techniques shown and have little practical understanding of how to apply them.

The good news is that modern machine learning can be distilled down to a couple of key techniques that are of very wide applicability. Recent studies have shown that the vast majority of datasets can be best modeled with just two methods:

- *Ensembles of decision trees* (i.e. Random Forests and Gradient Boosting Machines), mainly for structured data (such as you might find in a database table at most companies)
- *Multi-layered neural networks learnt with SGD* (i.e. shallow and/or deep learning), mainly for unstructured data (such as audio, vision, and natural language)

### ...this dataset

We will be looking at the Blue Book for Bulldozers Kaggle Competition: "The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuration.  The data is sourced from auction result postings and includes information on usage and equipment configurations."

This is a very common type of dataset and prediciton problem, and similar to what you may see in your project or workplace.

### ...Kaggle Competitions

Kaggle is an awesome resource for aspiring data scientists or anyone looking to improve their machine learning skills.  There is nothing like being able to get hands-on practice and receiving real-time feedback to help you improve your skills.

Kaggle provides:

1. Interesting data sets
2. Feedback on how you're doing
3. A leader board to see what's good, what's possible, and what's state-of-art.
4. Blog posts by winning contestants share useful tips and techniques.

## The data

### Look at the data

Kaggle provides info about some of the fields of our dataset; on the [Kaggle Data info](https://www.kaggle.com/c/bluebook-for-bulldozers/data) page they say the following:

For this competition, you are predicting the sale price of bulldozers sold at auctions. The data for this competition is split into three parts:

- **Train.csv** is the training set, which contains data through the end of 2011.
- **Valid.csv** is the validation set, which contains data from January 1, 2012 - April 30, 2012. You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
- **Test.csv** is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.

The key fields are in train.csv are:

- SalesID: the unique identifier of the sale
- MachineID: the unique identifier of a machine.  A machine can be sold multiple times
- saleprice: what the machine sold for at auction (only provided in train.csv)
- saledate: the date of the sale

Let's load in our data. We use the `read_csv` function in pandas. For very large datasets, you can supply the correct datatypes to use up front. This way, you won't run out of memory as easily.

We can also supply an indication of which columns contain datetime information with thje parameter `parse_dates`.

In [ ]:
dfs = [pd.read_csv(f'{PATH}{f}', low_memory=False) for f in data]

In [ ]:
# created_at             
# entry_id               
# TEMPERATURE            
# TURBIDITY              
# DISOLVED OXYGEN        
# pH                     
# AMMONIA                
# NITRATE                
# Population             
# Length                 
# Weight                 
# status                 
# Lenght
#                  
# Temperature (C)        
# Turbidity (NTU)        
# Dissolved Oxygen(g/ml) 
# PH                     
# Ammonia(g/ml)          
# Nitrate(g/ml)          
# Fish_Length (cm)       
# Fish_Weight (g)        
# Temperature(C)         
# Turbidity(NTU)         
# Fish_Length(cm)        
# Fish_Weight(g)         
# Date
#                    
# temperature(C)         
# turbidity (NTU)        
# Dissolved Oxygen (g/ml)
# ammonia(g/ml)          
# nitrate(g/ml)          
# Fish_length(cm)        
# Fish_weight(g)         
# Dissolved Oxygen (mg/L)
# Ammonia (mg/L)         
# Nitrate (mg/L)         
# Total_length (cm)      
# Weight (g)             
# DATE
# 
rename_columns_map = {
    'entry_id': 'entry_id',
    'TEMPERATURE': 'Temperature(C)',
    'temperature(C)': 'Temperature(C)',
    'Temperature (C)': 'Temperature(C)',
    'TURBIDITY': 'Turbidity(NTU)',
    'Turbidity (NTU)': 'Turbidity(NTU)',
    'turbidity (NTU)': 'Turbidity(NTU)',
    'DISOLVED OXYGEN': 'Dissolved_oxygen(g/ml)',
    'Dissolved Oxygen(g/ml)': 'Dissolved_oxygen(g/ml)',
    'Dissolved Oxygen (g/ml)': 'Dissolved_oxygen(g/ml)',
    'pH': 'PH',
    'AMMONIA': 'Ammonia(g/ml)',
    'ammonia(g/ml)': 'Ammonia(g/ml)',
    'NITRATE': 'Nitrate(g/ml)',
    'nitrate(g/ml)': 'Nitrate(g/ml)',
    'Population': 'Population',
    'Fish_Length(cm)': 'Fish_length(cm)',
    'Fish_Length (cm)': 'Fish_length(cm)',
    'Total_length (cm)': 'Fish_length(cm)',
    'Length': 'Fish_length(cm)',
    'Lenght': 'Fish_length(cm)',
    'Weight (g)': 'Fish_weight(g)',
    'Weight': 'Fish_weight(g)',
    'Fish_Weight(g)': 'Fish_weight(g)',
    'Fish_Weight (g)': 'Fish_weight(g)',
    'DATE': 'Date',
    'Dissolved Oxygen (mg/L)': 'Dissolved_oxygen(mg/L)',
    'Ammonia (mg/L)': 'Ammonia(mg/L)',
    'Nitrate (mg/L)': 'Nitrate(mg/L)',
}

In [ ]:
#dfs = dfs[4:]
df_raw = pd.concat(dfs)

In [ ]:
# convert g/ml to mg/L
df_raw['Dissolved_oxygen(g/ml)'] = df_raw['Dissolved_oxygen(g/ml)'] * 1000000
df_raw['Ammonia(g/ml)'] = df_raw['Ammonia(g/ml)'] * 1000000
df_raw['Nitrate(g/ml)'] = df_raw['Nitrate(g/ml)'] * 1000000

df_raw['Dissolved_oxygen(mg/L)'].fillna(df_raw['Dissolved_oxygen(g/ml)'], inplace=True)
df_raw['Ammonia(mg/L)'].fillna(df_raw['Ammonia(g/ml)'], inplace=True)
df_raw['Nitrate(mg/L)'].fillna(df_raw['Nitrate(g/ml)'], inplace=True)

# drop columns Dissolved_oxygen(g/ml), Ammonia(g/ml), Nitrate(g/ml)
df_raw.drop(['Dissolved_oxygen(g/ml)','Ammonia(g/ml)','Nitrate(g/ml)'], axis=1, inplace=True)


In [ ]:
# drop columns Date, entry_id, status, created_at
df_raw.drop(['Date','entry_id','status','created_at'], axis=1, inplace=True)

In [ ]:
# change type of column Fish_length(cm) to float
df_length = df_raw['Fish_length(cm)'].astype(float).infer_objects()

In any sort of data science work, it's **important to look at your data**, to make sure you understand the format, how it's stored, what type of values it holds, etc. Even if you've read descriptions about your data, the actual data may not be what you expect.

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
display_all(df_raw.head())

In [ ]:
display_all(df_raw.describe(include='all', datetime_is_numeric=True))

It's important to note what metric is being used for a project. Generally, selecting the metric(s) is an important part of the project setup. However, in this case Kaggle tells us what metric to use: RMSLE (root mean squared log error) between the actual and predicted auction prices. Therefore we take the log of the prices, so that RMSE will give us what we need.

### Initial processing

Using the random forest without any preprocessing will fail, because not all values are numeric.

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
# The following code is supposed to fail due to string values in the input data
m.fit(df_raw.drop('Fish_Weight(g)', axis=1), df_raw["Fish_Weight(g)"])

This dataset contains a mix of **continuous** and **categorical** variables. The **categorical** variables are still objects (strings) at this point, and not yet the actual "categorical" type.

In [ ]:
df_raw.dtypes

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals.  You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

In [ ]:
?? add_datepart

In [ ]:
add_datepart(df_raw, 'created_at')

Let's see what has been added to our dataframe with the `add_datepart()` function.

In [ ]:
display_all(df_raw.dtypes)

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call `train_cats` to convert strings to pandas categories.

In [ ]:
train_cats(df_raw)

In [ ]:
display_all(df_raw.dtypes)

We can specify the order to use for categorical variables if we wish:

In [ ]:
df_raw["UsageBand"].cat.categories

This might be useful for our Decision Trees to split, when the values have been made to be numeric.

In [ ]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. Optionally, we can replace the text categories with numbers, which will make this variable non-categorical, like so:.

In [ ]:
df_raw.UsageBand

We're still not quite done - for instance we have lots of missing values, which we can't pass directly to a random forest.

In [ ]:
display_all(df_raw.isnull().sum())

In [ ]:
display_all(df_raw.isnull().sum().sort_values(ascending=False)/len(df_raw))

But let's save this file for now, since it's already in format can we be stored and accessed efficiently.

In [ ]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/aquaponics.feather')

### Pre-processing

In the future we can simply read it from this fast format.

In [ ]:
df_raw = pd.read_feather('tmp/aquaponics.feather')

Now let's use fastai's `proc_df()` function. We'll replace categories with their numeric codes, handle missing values, and split the dependent variable into a separate variable.

For missing values, it does the following:
* For continuous variables, it checks whether a column has missing values or not
* If the column has missing values, it creates another column called columnname_na, which has 1 for missing and 0 for not missing
* Simultaneously, the missing values are replaced with the median of the column
* For categorical variables, pandas replaces missing values with -1. So proc_df adds 1 to all the values for categorical variables. Thus, we have 0 for missing while all other values are incremented by 1

In [ ]:
df_raw[df_raw.select_dtypes(np.float64).columns] = df_raw.select_dtypes(np.float64).astype(np.float32)
# drop entry_id column if exists
if 'entry_id' in df_raw.columns:
    df_raw.drop('entry_id', axis=1, inplace=True)

# drop rows with infinite values



In [ ]:
df, y, nas = proc_df(df_raw, 'Fish_Weight(g)')

In [ ]:
y

In [ ]:
display_all(df.dtypes)

In [ ]:
nas

In [ ]:
display_all(df_raw.tail())

In [ ]:
display_all(df.isnull().sum())

We now have something we can pass to a random forest!

In [ ]:
m = RandomForestRegressor(n_jobs=-1)

In [ ]:
%time m.fit(df, y)
m.score(df,y)

So what is that score?

In [ ]:
?m.score

In statistics, the coefficient of determination, denoted R2 or r2 and pronounced "R squared", is the proportion of the variance in the dependent variable that is predictable from the independent variable(s). https://en.wikipedia.org/wiki/Coefficient_of_determination

Wow, an r^2 of 0.98 - that's great, right? Well, perhaps not...

Possibly **the most important idea** in machine learning is that of having separate training & validation data sets. As motivation, suppose you don't divide up your data, but instead use all of it.  And suppose you have lots of parameters:

![Underfitting and Overfitting](https://i.stack.imgur.com/t0zit.png)

The error for the pictured data points is lowest for the model on the far right (the blue curve passes through the red points almost perfectly), yet it's not the best choice.  Why is that?  If you were to gather some new data points, they most likely would not be on that curve in the graph on the right, but would be closer to the curve in the middle graph.

This illustrates how using all our data can lead to **overfitting**. A validation set helps diagnose this problem.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=0.20, random_state=42)

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

# Random Forests

## Base model

Let's try our model again, this time with separate training and validation sets. And let's add some more metrics. Like RMSE (which is necessary to check our Kaggle rating, and an OOB score, which will be explained later on).

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = {'rmse_train': rmse(m.predict(X_train), y_train), 'rmse_valid': rmse(m.predict(X_valid), y_valid),
                'r2_train': m.score(X_train, y_train), 'r2_valid': m.score(X_valid, y_valid)}
    if hasattr(m, 'oob_score_'): res['r2_oob'] = m.oob_score_
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
%time m.fit(X_train, y_train)
print_score(m)

An r^2 in the high-80's isn't bad at all (and the RMSLE puts us around rank 100 of 470 on the Kaggle leaderboard), but we can see from the validation set score that we're over-fitting badly. To understand this issue, let's simplify things down to a single small tree.

## Speeding things up

Let's speed up things first, by selecting only a small subset of our entire dataset. This is always useful to do when you're still experimenting and tuning hyperparameters. Unless you have a total beast of a workstation of course. But even then, you might run into trouble with huge datasets.

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=10)
%time m.fit(X_train, y_train)
print_score(m)

Naturally, our results are a little worse, but we can still compare relative performance.

## Single tree

To understand what's going on under the hood, it's interesting to look at a single tree (parameter `n_estimators`). We restrict the `max_depth` to 3, otherwise we'd be staring straight into the mouth of madness.

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
draw_tree(m.estimators_[0], df_trn, precision=3)

Let's see what happens if we create a bigger tree.

In [ ]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

The training set result looks great! But the validation set is worse than our original model. This is why we need to use *bagging* of multiple trees to get more generalizable results.

## Bagging

### Intro to bagging

To learn about bagging in random forests, let's start with our basic model again.

In [ ]:
m = RandomForestRegressor(n_estimators = 10, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
y_valid[0]

We'll grab the predictions for each individual tree, and look at one example.

In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

These are the predictions from the different trees in our forest. The average of that (which is the final prediction), and the actual value from our validation data.

So how does this improve when we add more trees?

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(100)]);

The shape of this curve suggests that adding more trees isn't going to help us much. Let's check. (Compare this to our original model on a sample)

In [ ]:
m = RandomForestRegressor(n_estimators=20, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=80, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=1000, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

### Out-of-bag (OOB) score

Is our validation set worse than our training set because we're over-fitting, or because the validation set is for a different time period, or a bit of both? With the existing information we've shown, we can't tell. However, random forests have a very clever trick called *out-of-bag (OOB) error* which can handle this (and more!)

The idea is to calculate error on the training set, but only include the trees in the calculation of a row's error where that row was *not* included in training that tree. This allows us to see whether the model is over-fitting, without needing a separate validation set.

This also has the benefit of allowing us to see whether our model generalizes, even if we only have a small amount of data so want to avoid separating some out to create a validation set.

This is as simple as adding one more parameter to our model constructor. We print the OOB error last in our `print_score` function below.

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

This shows that our validation set time difference is making an impact, as is model over-fitting.

## Reducing over-fitting

### Subsampling

It turns out that one of the easiest ways to avoid over-fitting is also one of the best ways to speed up analysis: *subsampling*. Let's return to using our full dataset, so that we can demonstrate the impact of this technique.

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

The basic idea is this: rather than limit the total amount of data that our model can access (like we did before), let's instead limit it to a *different* random subset per tree. That way, given enough trees, the model can still see *all* the data, but for each individual tree it'll be just as fast as if we had cut down our dataset as before.



In [ ]:
m = RandomForestRegressor(n_jobs=-1, max_samples = 50000, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

Since each additional tree allows the model to see more data, this approach can make additional trees more useful.

In [ ]:
m = RandomForestRegressor(n_estimators=200, n_jobs=-1, max_samples = 50000, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

**IMPORTANT NOTE** 
Again, this technique is excellent to reduce training time. So use it for experimenting for sure. However, it remains to be seen if it actually increases the power of our model in the end. This is a trial-and-error process. At the end of your hyperparameter tuning, compare using `max_samples` with using the entire dataset.



### Tree building parameters

#### min_samples_leaf

Another way to reduce over-fitting is to grow our trees less deeply. We do this by specifying (with `min_samples_leaf`) that we require some minimum number of rows in every leaf node. This has two benefits:

- There are less decision rules for each leaf node; simpler models should generalize better
- The predictions are made by averaging more rows in the leaf node, resulting in less volatility

Some good values are: 3, 5, 10, 25, 100.

Let's get a baseline for this full set to compare to. With so few trees, OOB_score can't execute well, so we leave it out. For now, we're starting out with a basic model again, using 10 estimators, so the training process runs quickly.

In [ ]:
m = RandomForestRegressor(n_estimators=10, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

The function below calculates the max depth of the decision tree

In [ ]:
def dectree_max_depth(tree):
    children_left = tree.children_left
    children_right = tree.children_right

    def walk(node_id):
        if (children_left[node_id] != children_right[node_id]):
            left_max = 1 + walk(children_left[node_id])
            right_max = 1 + walk(children_right[node_id])
            return max(left_max, right_max)
        else: # leaf
            return 1

    root_node_id = 0
    return walk(root_node_id)

In [ ]:
t=m.estimators_[0].tree_

Let's check how deep our average standard tree is.

In [ ]:
dectree_max_depth(t)

How deep is it, when we set the minimum number of samples in the leaf node to 3?

In [ ]:
m = RandomForestRegressor(n_estimators=10, min_samples_leaf=3, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
t=m.estimators_[0].tree_

In [ ]:
dectree_max_depth(t)

#### max_features

We can also increase the amount of variation amongst the trees by not only using a sample of rows for each tree, but to also use a sample of *columns* for each *split*. We do this by specifying `max_features`, which is the proportion of features to randomly select from at each split.

- None
- 0.5
- 'sqrt'

In [ ]:
m = RandomForestRegressor(n_estimators=10, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

Now with 100 trees!

In [ ]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

Now with max_samples!

In [ ]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, max_features=0.5, max_samples = 50000, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

:-( seems worse, let's not use max_samples.

Check again how many trees are useful.

In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(100)]);

We can't compare our results directly with the Kaggle competition, since it used a different validation set (and we can no longer to submit to this competition) - but we can at least see that we're getting similar results to the winners based on the dataset we have.

The sklearn docs [show an example](http://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html) of different `max_features` methods with increasing numbers of trees - as you see, using a subset of features on each split requires using more trees, but results in better models:
![sklearn max_features chart](http://scikit-learn.org/stable/_images/sphx_glr_plot_ensemble_oob_001.png)